In [1]:
import os
import glob
import random

import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2


learning_rate = 0.001
batch_size = 10
epochs = 10

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)

In [2]:
train_path = "C:\\Users\\kssol\\PycharmProjects\\pp_lab5\\train"
test_path = "C:\\Users\\kssol\\PycharmProjects\\pp_lab5\\test"
val_path = "C:\\Users\\kssol\\PycharmProjects\\pp_lab5\\val"

path1 = "C:\\Users\\kssol\\PycharmProjects\\pp_lab5\\annotation_rose.csv"
path2 = "C:\\Users\\kssol\\PycharmProjects\\pp_lab5\\annotation_tulip.csv"
dfs = []
tmp = pd.read_csv(path1, sep=',', header=None, encoding='UTF-16') 
dfs.append(tmp)
tmp = pd.read_csv(path2, sep=',', header=None, encoding='UTF-16')
dfs.append(tmp)
df = pd.concat(dfs)
df.drop(1, axis=1, inplace=True)
df.rename(columns={0: 'absolute_way', 2: 'class_img'}, inplace=True)
df.reset_index(inplace=True)


if not os.path.isdir(train_path):
    os.mkdir(train_path)

if not os.path.isdir(test_path):
    os.mkdir(test_path)

if not os.path.isdir(val_path):
    os.mkdir(val_path)